# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [121]:
from os import getcwd
from numpy import exp, array, append, vectorize, mean, transpose
from numpy import float32
from scipy.optimize import minimize
from scipy.integrate import odeint
from plotnine import ggplot, aes, geom_point


In [130]:
YEAR = 0
DC14 = 1
SIG_DC14 = 2

In [131]:
with open(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv") as data:
    _ = next(data)  # Removing the header
    data = array([row.strip().split(",") for row in data], dtype=float32)
    data = transpose(data)

So for just modelling the troposphere I will want to set things up so that I have a differential equation where $f(x) = e^{x^{8}}$ is the rate of change. I will then use `scipy.optimize.minimize()` to minimize a loss function with respect to the solution of the ode as determined by the `return` of `ode.odeint`. 

In [143]:
def dydx(y, t, *params):
    def production(t, *params):
        peak, span, height = params  # Loading parameters
        return height * exp(-((t - peak) / (span / 2)) ** 8)   # Gaussian pulse
    return y + production(t, *params)

In [40]:
def loss(params=()):
    ode.odeint(dydx, mean(data["dc14"][:4]), data["year"], *params,)
    


In [144]:
odeint(func=dydx, y0=mean(data[DC14][:4]), t=data[YEAR], args=(775, 1, 5))

array([[-1.74500008e+01],
       [-1.28939040e+02],
       [-7.03983350e+03],
       [-5.20177264e+04],
       [-3.84361913e+05],
       [-2.84007183e+06],
       [-2.09854508e+07],
       [-1.55062679e+08],
       [-1.14576687e+09],
       [-8.46613601e+09],
       [-6.25567561e+10],
       [-4.62235396e+11],
       [-1.25648610e+12],
       [-3.41548339e+12],
       [-9.28424661e+12],
       [-2.52371993e+13],
       [-6.86018214e+13],
       [-1.86479088e+14],
       [-1.37790449e+15],
       [-1.01814139e+16],
       [-7.52310414e+16],
       [-5.55886405e+17],
       [-4.10747597e+18],
       [-3.03503715e+19],
       [-2.24260605e+20],
       [-1.65707425e+21],
       [-1.22442150e+22],
       [-9.04731949e+22],
       [-6.68511536e+23],
       [-4.93966942e+24],
       [-3.64994957e+25],
       [-2.69696831e+26],
       [-1.99280508e+27],
       [-1.47249491e+28],
       [-1.08803479e+29],
       [-8.03955035e+29],
       [-5.94046906e+30],
       [-4.38944607e+31]])

`pandas.read_csv`: 1.02 ms ± 65.6 µs

`list` with `append`: 57.5 µs ± 3.6 µs

`ndarray` with `append`: 668 µs ± 60.6 µs 

`ndarray` with `map`, `numpy.append`: 38.2 µs ± 4.07 µs

`ndarray` with `map`, `add_row`: 17.7 µs ± 1.34 µs 

`list` with `numpy.array` conversion at the end: 53.7 µs ± 3.14 µs